## Homework

In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

2024-12-02 05:52:27.962564: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 05:52:27.980369: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 05:52:28.000017: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 05:52:28.005962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-02 05:52:28.019697: I tensorflow/core/platform/cpu_feature_guar

### Reproducibility

In [2]:
tf.__version__

'2.17.0'

In [3]:
import numpy as np

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(200, 200, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

In [4]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),metrics=['acc'])

/opt/saturncloud/envs/saturn/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1733118751.513874   11454 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733118751.560659   11454 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733118751.560888   11454

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True,
)

Found 800 images belonging to 3 classes.


In [17]:
test_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True,
)

Found 202 images belonging to 3 classes.


In [11]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

/opt/saturncloud/envs/saturn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1733118774.316451   11494 service.cc:146] XLA service 0x7f8b8c0167b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733118774.316489   11494 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-02 05:52:54.348384: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-02 05:52:54.477936: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-12-02 05:52:54.900061: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 464.45MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 4/40 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - acc: 0.4302 - loss: -4.3487

I0000 00:00:1733118775.731328   11494 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - acc: 0.3611 - loss: nan 

2024-12-02 05:53:00.203078: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.53MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-12-02 05:53:00.252688: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 338.50MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 156ms/step - acc: 0.3569 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 123ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - a

In [18]:
accuracy = history.history['acc']
accuracy_median = np.median(accuracy)
accuracy_median

0.0

In [19]:
loss = history.history['loss']
loss_std = np.std(loss)
loss_std

nan

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.26
* 0.56
* 0.86
* 1.16

In [20]:
datagen = ImageDataGenerator(
        rotation_range=50,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
)

In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [25]:
train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 800 images belonging to 3 classes.


In [26]:
validation_generator = test_datagen.flow_from_directory(
    'data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary'
)

Found 202 images belonging to 3 classes.


In [27]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - acc: 0.0000e+00 - loss: nan

/opt/saturncloud/envs/saturn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 267ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 262ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 261ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 263ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 259ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 261ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 260ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 264ms/step - acc: 0.0000e+00 - loss: nan - val_acc: 0.0000e+00 - val_loss: nan
Epoch 9/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 2